# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [1]:
!pip install cirq --quiet
!pip install qutip --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 5.7 MB/s eta 

In [2]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [13]:
def oracle010(qq):
    yield X(qq[0])
    yield X(qq[2])
    yield Z(qq[2]).controlled_by(*(qq[0:2]))
    yield X(qq[0])
    yield X(qq[2])



In [14]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [15]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [16]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator()
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [17]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 94, '101': 2, '100': 1, '011': 1, '110': 1, '001': 1})


94

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring.

The function `oracle2` has

* input: `qq`, a 12-qubit register
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [18]:
def oracle2(qq):
    yield CX(qq[0],qq[5])
    yield CX(qq[3],qq[5])

    yield CX(qq[0],qq[6])
    yield CX(qq[4],qq[6])

    yield CX(qq[1],qq[7])
    yield CX(qq[3],qq[7])

    yield CX(qq[1],qq[8])
    yield CX(qq[4],qq[8])

    yield CX(qq[2],qq[9])
    yield CX(qq[3],qq[9])

    yield CX(qq[2],qq[10])
    yield CX(qq[4],qq[10])

    yield X(qq[11]).controlled_by(*(qq[5:11]))

In [19]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))

In [20]:
def grover2(trials_number):
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [21]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'11110': 4,
         '11100': 453,
         '00011': 454,
         '10110': 5,
         '01001': 3,
         '01000': 5,
         '00100': 5,
         '10010': 2,
         '11111': 2,
         '01101': 7,
         '10111': 4,
         '10000': 4,
         '11000': 6,
         '00101': 3,
         '00010': 1,
         '01110': 3,
         '00001': 3,
         '01010': 3,
         '00111': 2,
         '11011': 2,
         '00000': 3,
         '10001': 5,
         '01011': 6,
         '01111': 4,
         '11101': 1,
         '11001': 2,
         '10101': 3,
         '11010': 2,
         '10100': 1,
         '01100': 1,
         '10011': 1})

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [30]:
def oracle3(qq):

    yield CX(qq[0],qq[4])
    yield CX(qq[1],qq[4])

    yield CX(qq[0],qq[5])
    yield CX(qq[2],qq[5])

    yield CX(qq[0],qq[6])
    yield CX(qq[3],qq[6])

    yield CX(qq[1],qq[7])
    yield CX(qq[2],qq[7])

    yield CX(qq[1],qq[8])
    yield CX(qq[3],qq[8])

    yield CX(qq[4],qq[9])

    # add qubits 5-7
    for j in range(5,8):
        yield CCX(qq[j], qq[9], qq[10])
        yield CX(qq[j], qq[9])

    # add qubits 8
    yield X(qq[11]).controlled_by(qq[8], qq[9], qq[10])
    yield CCX(qq[8], qq[9], qq[10])
    yield CX(qq[8], qq[9])

    # check if number is equal to four
    yield X(qq[9])
    yield X(qq[10])
    yield X(qq[12]).controlled_by(*(qq[8:11]))






In [31]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))

In [32]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX

def grover3(trials_number):
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [33]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'1100': 482,
         '0011': 464,
         '0001': 3,
         '1011': 5,
         '0000': 4,
         '1001': 4,
         '0100': 4,
         '0101': 6,
         '1010': 2,
         '0111': 3,
         '1111': 4,
         '0010': 3,
         '1000': 2,
         '1110': 3,
         '1101': 6,
         '0110': 5})

In [ ]:
# hidden tests in this cell will be used for grading.